<a href="https://colab.research.google.com/github/SamuelJucker/ML_juckesam/blob/master/custom_finnews_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preparation:**

Install the following libraries

In [1]:
!pip install transformers datasets
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x8

Necessary imports // if not acess to drive just comment it out

In [2]:

# Import necessary libraries
import transformers
import accelerate
import torch
import shutil
import os
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, GenerationConfig
from datasets import load_dataset
from google.colab import drive

import random  # Add this line to import the random module
import pandas as pd  # Ensure pandas is imported for DataFrame usage
from datasets import Dataset




loading / model and tokenizer

In [3]:
# # # Load the model and tokenizer
# # model_name = "google/pegasus-large"  # Using a more powerful model
# # tokenizer = AutoTokenizer.from_pretrained(model_name)
# # model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# # Load the model and tokenizer
# model_name = "google/pegasus-large"  # Using a more powerful model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# base_model = model_name.split('/')[1]  # Extract the base model name for later use



# # Define the directory to save the model and tokenizer
# save_directory = "./pretrained_pegasus_large"

# # Save the model and tokenizer to the directory
# tokenizer.save_pretrained(save_directory)
# model.save_pretrained(save_directory)

# # Print all files in the save directory
# print("Files in the save directory:")
# for root, dirs, files in os.walk(save_directory):
#     for filename in files:
#         print(os.path.join(root, filename))

#   # Assuming you have already fine-tuned the model and tokenizer
# model.save_pretrained('./results')
# tokenizer.save_pretrained('./results')

# # Optionally, save the generation configuration if used
# gen_config = GenerationConfig(
#     max_length=142,
#     min_length=56,
#     early_stopping=True,
#     num_beams=4,
#     length_penalty=2.0,
#     no_repeat_ngram_size=3,
#     forced_bos_token_id=0,
#     forced_eos_token_id=2
# )
# gen_config.save_pretrained('./results')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig
import shutil
import os

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the Save Path
drive_save_path = '/content/drive/My Drive/fine_tuned_model'

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = model_name.split('/')[1]  # Extract the base model name for later use

# Define the directory to save the model and tokenizer
save_directory = "./pretrained_pegasus_large"

# Save the pretrained model and tokenizer to the directory
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

# Print all files in the save directory
print("Files in the save directory:")
for root, dirs, files in os.walk(save_directory):
    for filename in files:
        print(os.path.join(root, filename))

# Assuming you have already fine-tuned the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Optionally, save the generation configuration if used
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained('./results')

# List of files to keep (fine-tuned files)
files_to_keep = [
    "config.json",
    "generation_config.json",
    "model.safetensors",  # or 'pytorch_model.bin'
    "special_tokens_map.json",
    "spiece.model",
    "tokenizer.json",
    "tokenizer_config.json"
]

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk('./results'):
    for filename in files:
        print(os.path.join(root, filename))

# Step 4: Copy Only Fine-Tuned Files to Google Drive
# If the directory exists, delete it first
if os.path.exists(drive_save_path):
    shutil.rmtree(drive_save_path)

# Create the directory in Google Drive
os.makedirs(drive_save_path)

# Copy only the necessary files to Google Drive
for file_name in files_to_keep:
    src_path = os.path.join('./results', file_name)
    dest_path = os.path.join(drive_save_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")
    else:
        print(f"{src_path} does not exist")

# Print all files in the Google Drive save directory to ensure they are saved
print("Files in the Google Drive save directory:")
for root, dirs, files in os.walk(drive_save_path):
    for filename in files:
        print(os.path.join(root, filename))



Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


Files in the save directory:
./pretrained_pegasus_large/config.json
./pretrained_pegasus_large/model.safetensors
./pretrained_pegasus_large/tokenizer.json
./pretrained_pegasus_large/spiece.model
./pretrained_pegasus_large/special_tokens_map.json
./pretrained_pegasus_large/tokenizer_config.json
./pretrained_pegasus_large/generation_config.json
Files in the fine-tuned directory:
./results/config.json
./results/model.safetensors
./results/tokenizer.json
./results/spiece.model
./results/special_tokens_map.json
./results/tokenizer_config.json
./results/generation_config.json
Copied ./results/config.json to /content/drive/My Drive/fine_tuned_model/config.json
Copied ./results/generation_config.json to /content/drive/My Drive/fine_tuned_model/generation_config.json
Copied ./results/model.safetensors to /content/drive/My Drive/fine_tuned_model/model.safetensors
Copied ./results/special_tokens_map.json to /content/drive/My Drive/fine_tuned_model/special_tokens_map.json
Copied ./results/spiece.m

take a look at this: https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model

Dataload / data preproc etc not in collab

# Define the dataset name
dataset_name = "this can vary"


In [4]:
# dataset_name = "cnn_dailymail"
# dataset_version = "3.0.0"


Load temporary dataset, currently temp future final temp down.

In [5]:
import json
import pandas as pd

train_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_train.jsonl'
val_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_val.jsonl'
test_json_file_path = '/content/drive/My Drive/Data_ML2/ECTSum_test.jsonl'

# Set the dataset name
dataset_short_name = 'ECTSum'

# Read the JSONL files
train_data = []
with open(train_json_file_path, 'r') as file:
    for line in file:
        train_data.append(json.loads(line))

val_data = []
with open(val_json_file_path, 'r') as file:
    for line in file:
        val_data.append(json.loads(line))

test_data = []
with open(test_json_file_path, 'r') as file:
    for line in file:
        test_data.append(json.loads(line))

# Convert to DataFrames for better handling
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Combine the datasets for preprocessing and fine-tuning
combined_df = pd.concat([train_df, val_df, test_df], ignore_index=True)


# Retrieve a random sample
random_index = random.randint(0, len(combined_df) - 1)
initial_sample = combined_df.iloc[random_index]
print("Random sample from the initial dataset (before processing):")
print(initial_sample)
#Load the dataset
dataset = Dataset.from_pandas(combined_df)

# Load the tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a preprocessing function
# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
# Apply the preprocessing function to the dataset

# Define a preprocessing function
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_function, batched=True)

# Retrieve the processed sample
processed_sample = processed_dataset[random_index]
print("\nRandom sample from the processed dataset (after processing):")
print(processed_sample)
# dataset = load_dataset(dataset_name, dataset_version)

# random_index = random.randint(0, len(dataset["train"]) - 1)
# initial_sample = dataset["train"][random_index]
# print("Random sample from the initial dataset:")
# print(pd.DataFrame([initial_sample]))

# train_dataset = dataset["train"].select(range(500))  # Increase subset size for more data
# validation_dataset = dataset["validation"].select(range(1000))  # Increase subset size for validation
# test_dataset = dataset["test"]

# dataset_short_name = dataset_name[:5]  # Get the first five letters of the dataset name


# # # Load the dataset
# # dataset = load_dataset(dataset_name, "3.0.0") # specifiy version manually
# # #sample before:
# # random_index = random.randint(0, len(dataset["train"]) - 1)
# # initial_sample = dataset["train"][random_index]
# # print("Random sample from the initial dataset:")
# # print(pd.DataFrame([initial_sample]))


# # train_dataset = dataset["train"].select(range(500))  # Increase subset size for more data
# # validation_dataset = dataset["validation"].select(range(1000))  # Increase subset size for validation
# # test_dataset = dataset["test"]


Random sample from the initial dataset (before processing):
ticker                                                   CBT
quarter                                                   q4
year                                                    2021
input      Last night,we released results for our fourth ...
summary    compname reports fourth quarter diluted earnin...
Name: 219, dtype: object


Map:   0%|          | 0/2425 [00:00<?, ? examples/s]


Random sample from the processed dataset (after processing):
{'ticker': 'CBT', 'quarter': 'q4', 'year': '2021', 'input': "Last night,we released results for our fourth quarter of fiscal year 2021, copies of which are posted in the Investor Relations section of our website.\nIn order to provide greater transparency regarding our operating performance, we refer to certain non-GAAP financial measures that involve adjustments to GAAP results.\nAny non-GAAP financial measures presented should not be considered to be an alternative to financial measures required by GAAP.\nErica will review the business segment and corporate financial details.\nAt this time last year.\nI talked about being prepared to win as the recovery takes hold and I think 2021 is evidence that we were prepared.\nFiscal 2021 was a remarkable year for Cabot as the business recovered from the 2020 COVID lows to deliver record results.\nWe accomplished this through a combination of extraordinary execution and our commitment

In [6]:
# # Load the dataset

# dataset = load_dataset("cnn_dailymail", "3.0.0")
# train_dataset = dataset["train"].select(range(1900))  # Increase subset size for more data

# #smaller dataset chosen for practical reason final dataset was:

# #train_dataset = dataset["train"].select(range(500))  # Increase subset size for more data
# validation_dataset = dataset["validation"].select(range(1100))  # Increase subset size for validation
# test_dataset = dataset["test"]


Preprocess Data

In [7]:
# Preprocess the data
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")  # Reduced max_length
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)


NameError: name 'train_dataset' is not defined

sample after

In [ ]:

# Inspect a random sample from the preprocessed dataset
random_index = random.randint(0, len(train_dataset) - 1)
preprocessed_sample = train_dataset[random_index]
print("Random sample from the preprocessed dataset:")
print(pd.DataFrame([preprocessed_sample]))

In [ ]:
# # Preprocess the data
# def preprocess_function(examples):
#     inputs = [doc for doc in examples["article"]]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")  # Reduced max_length
#     labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# validation_dataset = validation_dataset.map(preprocess_function, batched=True)
# test_dataset = test_dataset.map(preprocess_function, batched=True)


make length etc bigger for final version

In [ ]:
# Use DataCollatorForSeq2Seq to handle padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


## **training**

In [ ]:
# Define training arguments with mixed precision and gradient accumulation
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust batch size if necessary
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Increase number of epochs for better learning
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
    logging_steps=100,  # Reduce logging frequency
    save_steps=500,  # Adjust save steps
    evaluation_strategy="steps",  # Evaluate every save step
    eval_steps=500,  # Adjust evaluation steps
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)


In [ ]:
# #done for project: (just better more computing power)
# # Define training arguments with mixed precision and gradient accumulation
# training_args = TrainingArguments(
#     output_dir="./results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,  # Adjust batch size if necessary
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,  # Increase number of epochs for better learning
#     weight_decay=0.01,
#     fp16=True,  # Enable mixed precision training
#     gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps
#     logging_steps=100,  # Reduce logging frequency
#     save_steps=500,  # Adjust save steps
#     evaluation_strategy="steps",  # Evaluate every save step
#     eval_steps=500,  # Adjust evaluation steps
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
# )


In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
)


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


checks:

to do parameter finetuning and: Instruction Tuning is Key: For zero-shot summarization tasks, instruction tuning significantly outperforms mere model scaling.

In [ ]:
# Ensure model is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Train

wtf comp to other

In [ ]:
# Train the model
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset,  # Using the same dataset for simplicity; you should ideally have a separate validation dataset
    data_collator=data_collator,
)
trainer.train()


Vorlage:

In [ ]:
import shutil
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the Save Path
drive_save_path = '/content/drive/My Drive/fine_tuned_model'

# Load the pretrained model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Assume `model` is your fine-tuned model and `tokenizer` is your tokenizer
# Define the local directory to save the fine-tuned model and tokenizer
fine_tuned_directory = "./results"

# Save the fine-tuned model and tokenizer to the local directory
model.save_pretrained(fine_tuned_directory)
tokenizer.save_pretrained(fine_tuned_directory)

# Optionally, save the generation configuration if used
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained(fine_tuned_directory)
# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

# List of files to keep (fine-tuned files)
files_to_keep = [
    "config.json",
    "generation_config.json",
    "model.safetensors",  # or 'pytorch_model.bin'
    "special_tokens_map.json",
    "spiece.model",
    "tokenizer.json",
    "tokenizer_config.json"
]

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

# Step 4: Copy Only Fine-Tuned Files to Google Drive
# If the directory exists, delete it first
if os.path.exists(drive_save_path):
    shutil.rmtree(drive_save_path)

# Create the directory in Google Drive
os.makedirs(drive_save_path)

# Copy only the necessary files to Google Drive
for file_name in files_to_keep:
    src_path = os.path.join(fine_tuned_directory, file_name)
    dest_path = os.path.join(drive_save_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, dest_path)
        print(f"Copied {src_path} to {dest_path}")
    else:
        print(f"{src_path} does not exist")

# Print all files in the Google Drive save directory to ensure they are saved
print("Files in the Google Drive save directory:")
for root, dirs, files in os.walk(drive_save_path):
    for filename in files:
        print(os.path.join(root, filename))


In [ ]:
# import shutil

# # Define directories
# pretrained_directory = "./pretrained_pegasus_large"
# fine_tuned_directory = "./results"

# # List of files to copy
# files_to_copy = [
#     "config.json",
#     "generation_config.json",
#     "model.safetensors",  # or 'pytorch_model.bin'
#     "special_tokens_map.json",
#     "spiece.model",
#     "tokenizer.json",
#     "tokenizer_config.json"
# ]

# # Copy files from the pretrained directory to the fine-tuned directory
# for file_name in files_to_copy:
#     src_path = os.path.join(pretrained_directory, file_name)
#     dest_path = os.path.join(fine_tuned_directory, file_name)
#     if os.path.exists(src_path):
#         shutil.copy(src_path, dest_path)
#         print(f"Copied {src_path} to {dest_path}")
#     else:
#         print(f"{src_path} does not exist")


Imports:




move up Check your imports. expected results:
Transformers version: 4.41.2
Accelerate version: 0.31.0
Torch version: 2.3.0+cu121

In [ ]:
# Check the versions of transformers, accelerate, and torch
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
print("Torch version:", torch.__version__)


**evaluation**

In [ ]:
# Evaluate the model (short overvie numbers)
metrics = trainer.evaluate(eval_dataset=test_dataset)
print(metrics)


Save Configuaration from finetiuned /tempraus

In [ ]:
# # Define and save generation configuration
# gen_config = GenerationConfig(
#     max_length=142,
#     min_length=56,
#     early_stopping=True,
#     num_beams=4,
#     length_penalty=2.0,
#     no_repeat_ngram_size=3,
#     forced_bos_token_id=0,
#     forced_eos_token_id=2
# )
# gen_config.save_pretrained('./results')

Libraries for evaluation

EVALUATION CURRENTLY WRONG

In [ ]:
!pip install rouge_score evaluate

Try Out:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example text
text = """
The fake industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=142, min_length=56, early_stopping=True, num_beams=4, length_penalty=2.0, no_repeat_ngram_size=3)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


try out with paths

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the pretrained model and tokenizer
model_name = "google/pegasus-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example text
text = """
The fake industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=142, min_length=56, early_stopping=True, num_beams=4, length_penalty=2.0, no_repeat_ngram_size=3)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

# Print all files in the current directory
print("Files in the current directory:")
for root, dirs, files in os.walk("."):
    for filename in files:
        print(os.path.join(root, filename))


In [ ]:
import os

# Print all files in the results directory
print("Files in the results directory:")
for root, dirs, files in os.walk("./results"):
    for filename in files:
        print(os.path.join(root, filename))


again weights problem and shit

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig

# Load the fine-tuned model and tokenizer
model_directory = "./results"  # Path to your fine-tuned model directory
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
gen_config = GenerationConfig.from_pretrained(model_directory)

# Set decoder_start_token_id if not already set
if gen_config.decoder_start_token_id is None:
    gen_config.decoder_start_token_id = model.config.decoder_start_token_id
if gen_config.bos_token_id is None:
    gen_config.bos_token_id = model.config.bos_token_id

# Example text
text = """
The Industrial Revolution was a period of major industrialization and innovation that took place during the late 1700s and early 1800s. It began in Great Britain and quickly spread throughout the world. This period marked a significant turning point in history; almost every aspect of daily life was influenced in some way. Improvements in farming techniques and livestock breeding led to increased food production and the rise of the steam engine. The rise of factories created a new class of wealthy industrialists and businessmen, but also created social tensions and unrest. Despite many challenges and hardships, the Industrial Revolution was a period of significant change and transformation, paving the way for the modern industrial economy.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary using fine-tuned model
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=gen_config.max_length,
    min_length=gen_config.min_length,
    early_stopping=gen_config.early_stopping,
    num_beams=gen_config.num_beams,
    length_penalty=gen_config.length_penalty,
    no_repeat_ngram_size=gen_config.no_repeat_ngram_size,
    forced_bos_token_id=gen_config.forced_bos_token_id,
    forced_eos_token_id=gen_config.forced_eos_token_id,
    decoder_start_token_id=gen_config.decoder_start_token_id,
    bos_token_id=gen_config.bos_token_id
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


In [ ]:
# # Copy the zipped model to Google Drive with appropriate naming
# def copy_to_drive_with_naming(base_name, drive_path):
#     file_index = 0
#     while True:
#         file_name = f"{base_name}{file_index}.zip"
#         destination_path = os.path.join(drive_path, file_name)
#         if not os.path.exists(destination_path):
#             shutil.copyfile('model.zip', destination_path)
#             print(f"Model saved as: {file_name}")
#             break
#         file_index += 1

# copy_to_drive_path = "/content/drive/My Drive/"
# copy_to_drive_with_naming(base_name, copy_to_drive_path)


do some kind of check / just numbers not whole valuation

Finishing Project Steps / Model Storage etc: no need to execute

In [ ]:
# Save the model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')
gen_config = GenerationConfig(
    max_length=142,
    min_length=56,
    early_stopping=True,
    num_beams=4,
    length_penalty=2.0,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)
gen_config.save_pretrained(fine_tuned_directory)

# Print all files in the fine-tuned directory to ensure they are saved
print("Files in the fine-tuned directory:")
for root, dirs, files in os.walk(fine_tuned_directory):
    for filename in files:
        print(os.path.join(root, filename))

In [ ]:
# Zip the results directory, including only direct files
shutil.make_archive('model', 'zip', './results', './')


In [ ]:
# Upload the zipped model to Google Drive
drive.mount('/content/drive')


In [ ]:

# Copy the zipped model to Google Drive with appropriate naming
def copy_to_drive_with_naming(base_name, drive_path):
    file_index = 0
    while True:
        file_name = f"{base_name}{file_index}.zip"
        destination_path = os.path.join(drive_path, file_name)
        if not os.path.exists(destination_path):
            shutil.copyfile('model.zip', destination_path)
            print(f"Model saved as: {file_name}")
            break
        file_index += 1

# Extract the base model name and dataset short name to use as part of the base name
base_name = f"model_{base_model}_{dataset_short_name}_"
copy_to_drive_path = "/content/drive/My Drive/Models_ML2"
copy_to_drive_with_naming(base_name, copy_to_drive_path)




# # Copy the zipped model to Google Drive with appropriate naming
# def copy_to_drive_with_naming(base_name, drive_path):
#     file_index = 0
#     while True:
#         file_name = f"{base_name}{file_index}.zip"
#         destination_path = os.path.join(drive_path, file_name)
#         if not os.path.exists(destination_path):
#             shutil.copyfile('model.zip', destination_path)
#             print(f"Model saved as: {file_name}")
#             break
#         file_index += 1

# # Extract the base model name to use as part of the base name
# base_name = f"model_{base_model}_"
# copy_to_drive_path = "/content/drive/My Drive/"
# copy_to_drive_with_naming(base_name, copy_to_drive_path)



# # Copy the zipped model to Google Drive with appropriate naming
# def copy_to_drive_with_naming(base_name, drive_path):
#     file_index = 0
#     while True:
#         file_name = f"{base_name}{file_index}.zip"
#         destination_path = os.path.join(drive_path, file_name)
#         if not os.path.exists(destination_path):
#             shutil.copyfile('model.zip', destination_path)
#             print(f"Model saved as: {file_name}")
#             break
#         file_index += 1

# # Extract the model name to use as part of the base name
# base_name = f"model_{model_name.replace('/', '_')}_"
# copy_to_drive_path = "/content/drive/My Drive/"
# copy_to_drive_with_naming(base_name, copy_to_drive_path)

In [ ]:
# # Copy the zipped model to Google Drive with appropriate naming
# def copy_to_drive_with_naming(base_name, drive_path):
#     file_index = 0
#     while True:
#         file_name = f"{base_name}{file_index}.zip"
#         destination_path = os.path.join(drive_path, file_name)
#         if not os.path.exists(destination_path):
#             shutil.copyfile('model.zip', destination_path)
#             print(f"Model saved as: {file_name}")
#             break
#         file_index += 1

# copy_to_drive_path = "/content/drive/My Drive/"
# base_name = "model_google-pegasus-large_"
# copy_to_drive_with_naming(base_name, copy_to_drive_path)


# Zip the results directory, including only direct files
shutil.make_archive('model', 'zip', './results', './')


Examples

Vergleiche/ Validierungen